In [209]:
import pandas as pd
import numpy as np
import sys
import os
from bs4 import BeautifulSoup
from IPython.display import display, HTML
from datetime import datetime
import re

In [210]:
IMPORT_FILE = 'Scouting_Dump_Oct_2025.html'

In [211]:
def open_dir(dir_name):
    os.chdir(dir_name)
    os.getcwd()

In [212]:
def parse_html_table(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    table = soup.find('table')  # Adjust the selector if needed

    # Find headers and data rows directly from the table
    headers = [th.text for th in table.find_all('th')]
    data = []
    for row in table.find_all('tr')[1:]:  # Skip the first row if it's a header
        row_data = [td.text for td in row.find_all('td')]
        data.append(row_data)

    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)
    return df

In [213]:
os.getcwd()

'C:\\Users\\lover\\OneDrive\\Documents\\GitHub\\football_manager_code\\AI'

In [214]:
scouting_data = parse_html_table(IMPORT_FILE)
scouting_data.head()

,Inf,Name,UID,Best Pos,Best Role,Best Duty,Pros,Cons,Position,Club,...,Cmp,Com,Cmd,Bra,Bal,Ant,Agi,Agg,Aer,Acc
0,,,,,,,,,,,...,,,,,,,,,,
1,Wnt,Martial Tia,12095307,M (C),Central Midfielder,Attack,Speed,Marking,M/AM (C),AFE,...,10,3,2,7,8,7,11,14,1,13
2,,Ibrahim Sulley,13193986,AM (C),Shadow Striker,Attack,Speed,Team Player,"AM (C), ST (C)",Great Olympics,...,9,3,2,6,8,9,14,12,1,14
3,Loa,Mohamed Hamdy,2000071475,AM (C),Advanced Playmaker,Support,Intelligence,Athleticism,M/AM (C),Al-Ahly,...,11,1,2,8,9,11,13,8,4,10
4,,George Ayine,83258606,M (C),Ball Winning Midfielder,Defend,Speed,Unknown,M/AM (C),Fréjus-Saint-Raphaël,...,9,2,2,11,9,11,10,12,2,12


In [215]:
scouting_data.columns

Index(['Inf', 'Name', 'UID', 'Best Pos', 'Best Role', 'Best Duty', 'Pros',
       'Cons', 'Position', 'Club',
       ...
       'Cmp', 'Com', 'Cmd', 'Bra', 'Bal', 'Ant', 'Agi', 'Agg', 'Aer', 'Acc'],
      dtype='object', length=203)

In [216]:
for col in scouting_data.columns:
    print(col)

Inf
Name
UID
Best Pos
Best Role
Best Duty
Pros
Cons
Position
Club
On Loan From
Based
Asking Price
Salary
Expires
Nat
2nd Nat
Home Grown Status
Height
Weight
Preferred Foot
Age
Personality
Media Handling
Transfer Value
Apps
Starts
Mins
Mins/Gm
Av Rat
Yth Gls
Yth Apps
Gls
Gls/90
Shots
ShT
Shot %
Shot/90
ShT/90
Pens
Pens S
Pen/R
Off
xG
CCC
Ch C/90
Ast
Asts/90
K Pas
K Ps/90
Pas A
Ps A/90
Ps C
Ps C/90
Pas %
Cr A
Cr C
Cr C/A
Drb
Goals Outside Box
FK Shots
OP-Cr %
Tck A
Shots Outside Box/90
Shts Blckd/90
Shts Blckd
Pr passes/90
Pr Passes
Pres C/90
Pres C
Pres A/90
Pres A
Poss Won/90
Poss Lost/90
OP-KP/90
OP-KP
OP-Crs C/90
OP-Crs C
OP-Crs A/90
OP-Crs A
Gl Mst
Hdrs L/90
K Tck/90
K Hdrs/90
Sprints/90
xGP/90
Blk
xG/shot
xGP
Clr/90
Blk/90
Conv %
Crs A/90
Cr C/90
Dist/90
Drb/90
FA
Distance
Hdrs A
Aer A/90
Hdrs
Hdrs W/90
Hdr %
K Tck
Tck W
Tck/90
Tck R
Fls
Yel
Red
Itc
Int/90
Clear
Conc
All/90
Shutouts
xSv %
Svh
Svt
Svp
Sv %
Pens Faced
Pens Saved
Pens Saved Ratio
PoM
AT Apps
Saves/90
Tgls/90
Tcon/90
T

We need to exclude some columns and the rest we need to cast into numerical data and/or clean accordingly.

In [217]:
EXCLUDE_COLS = """Inf
Name
Apps
Best Pos
Best Role
Best Duty
Pros
Expires
Distance
Dist/90
Cons
Position
Club
On Loan From
Based
Asking Price
Salary
Nat
2nd Nat
Home Grown Status
Height
Weight
Preferred Foot
Personality
Media Handling
Shot %
Pas %
OP-Cr %
Conv %
Hdr %
xSv %
Sv %
Transfer Value""".split("\n")
EXCLUDE_COLS

['Inf',
 'Name',
 'Apps',
 'Best Pos',
 'Best Role',
 'Best Duty',
 'Pros',
 'Expires',
 'Distance',
 'Dist/90',
 'Cons',
 'Position',
 'Club',
 'On Loan From',
 'Based',
 'Asking Price',
 'Salary',
 'Nat',
 '2nd Nat',
 'Home Grown Status',
 'Height',
 'Weight',
 'Preferred Foot',
 'Personality',
 'Media Handling',
 'Shot %',
 'Pas %',
 'OP-Cr %',
 'Conv %',
 'Hdr %',
 'xSv %',
 'Sv %',
 'Transfer Value']

In [218]:
# EXCLUDE_COLS.remove('Age')  # Not a text col
# EXCLUDE_COLS.remove('UID')  # UUID
# EXCLUDE_COLS.remove('Expires')  # Should convert to Datetime
EXCLUDE_COLS.append('decimal_height')  # Should convert to Datetime


In [219]:
EXCLUDE_COLS

['Inf',
 'Name',
 'Apps',
 'Best Pos',
 'Best Role',
 'Best Duty',
 'Pros',
 'Expires',
 'Distance',
 'Dist/90',
 'Cons',
 'Position',
 'Club',
 'On Loan From',
 'Based',
 'Asking Price',
 'Salary',
 'Nat',
 '2nd Nat',
 'Home Grown Status',
 'Height',
 'Weight',
 'Preferred Foot',
 'Personality',
 'Media Handling',
 'Shot %',
 'Pas %',
 'OP-Cr %',
 'Conv %',
 'Hdr %',
 'xSv %',
 'Sv %',
 'Transfer Value',
 'decimal_height']

In [220]:
def imperial_to_decimal_height(height: str) -> float:
    # Check if the height is a valid string and in the correct format
    if isinstance(height, str) and "'" in height and '"' in height:
        try:
            # Split into feet and inches
            feet, inches = height.split("'")
            feet = int(feet)
            inches = int(inches.replace('"', ''))
        except ValueError:
            # Return NaN if the height format is invalid
            return np.nan

        # Convert to decimal format
        return round(feet + inches / 12, 2)
    
    # Return NaN if the value is not a valid string or doesn't match the format
    return np.nan


In [221]:
def weight_to_decimal_weight(weight: str) -> float:
    # Check if the height is a valid string and in the correct format
    if isinstance(weight, str) and "lbs" in weight:
        try:
            # Split into feet and inches
            weight, lbs_str = weight.split(" ")
            weight = int(weight)
        except ValueError:
            # Return NaN if the height format is invalid
            return np.nan

        # Convert to decimal format
        return round(weight)
    
    # Return NaN if the value is not a valid string or doesn't match the format
    return np.nan


In [222]:
def convert_percent_string_to_decimal(row, col):
    percent_string = row[col]
    try:
        percent_string = percent_string.split("%")[0]
        percent_int = int(percent_string)
        percent_value = percent_int / 100
        return percent_value
    except (ValueError, TypeError) as e:
        print(f"Error processing result for row {row[col]}: {e}")
        return None

# scouting_df['Conv %'] = scouting_df.apply(lambda row: convert_percent_string_to_decimal(row, 'Conv %'), axis=1)
    

In [223]:
# Define a function that attempts to convert a column to numeric
def convert_to_numeric(df, column):
    try:
        # Convert to numeric
        df[column.name] = pd.to_numeric(column.str.replace(',',''), errors='coerce')  # Convert non-numeric to NaN
        # df[column] = pd.to_numeric(df[column].str.replace(',', ''), errors='coerce')  # Convert non-numeric to NaN
        print(f"Converted {column.name} to numeric")
    except Exception as e:
        print(f"Error processing {column.name}: {e}")
        
# club_info_data.dtypes
# convert_to_numeric(club_info_data, club_info_data['estimated_value'])

In [224]:
def determine_goal_differential(row, flag='for'):
    initial_result = row['result']
    # print(f"Type of row['result']: {type(row['result'])}")
    # print(f"Result: {initial_result}")
    try:
        home, away = initial_result.strip(" ").split("-")
        home = int(home)
        away = int(away)

#         print(f"Home: {home} Away: {away}")
#         print(f"Venue: {row['venue']}")
        
        score_map = {'home': { 'for': home, 'against': away },
                     'away': { 'for': away, 'against': home }}

        return score_map[row['venue']][flag]
    except (ValueError, TypeError) as e:
        print(f"Error processing result for row {row['index']}: {e}")
        return None 
    

# match_df['gf'] = match_df.apply(lambda row: determine_goal_differential(row, flag='for'), axis=1)

In [225]:
def determine_expires_date(row):
    initial_date = row['Expires']
    try:
        return datetime.strptime(initial_date, "%m/%d/%Y")
    except:
        return None
    
# lambda x: datetime.strptime(x, "%m/%d/%Y")

In [226]:
# scouting_data['Height'].head()
# print(f"Starting {len(scouting_data)}")
# scouting_data.dropna()
# print(f"Ending {len(scouting_data)}")

scouting_data.dtypes

Inf          object
Name         object
UID          object
Best Pos     object
Best Role    object
              ...  
Ant          object
Agi          object
Agg          object
Aer          object
Acc          object
Length: 203, dtype: object

In [227]:
# Defragmentation routine
scouting_data = scouting_data.copy()
# Convert text height to decimal height
scouting_data['decimal_height'] = scouting_data['Height'].apply(imperial_to_decimal_height)

# Convert weight similarly 
scouting_data['decimal_weight'] = scouting_data['Weight'].apply(weight_to_decimal_weight)

In [228]:
scouting_data['decimal_weight']

0        NaN
1      154.0
2      132.0
3      127.0
4      167.0
       ...  
252    178.0
253    154.0
254    145.0
255    134.0
256    134.0
Name: decimal_weight, Length: 257, dtype: float64

In [229]:
#scouting_data['decimal_height'].dtype
# scouting_data['Name'].dtype == object

Try to convert all cols not in our `EXCLUDE_COLS` to numeric. 

In [230]:
for col in scouting_data.columns:
    if col not in EXCLUDE_COLS:
        try:
            print(f"Processing {col}")
            convert_to_numeric(scouting_data, scouting_data[col])
        except:
            pass


Processing UID
Converted UID to numeric
Processing Age
Converted Age to numeric
Processing Starts
Converted Starts to numeric
Processing Mins
Converted Mins to numeric
Processing Mins/Gm
Converted Mins/Gm to numeric
Processing Av Rat
Converted Av Rat to numeric
Processing Yth Gls
Converted Yth Gls to numeric
Processing Yth Apps
Converted Yth Apps to numeric
Processing Gls
Converted Gls to numeric
Processing Gls/90
Converted Gls/90 to numeric
Processing Shots
Converted Shots to numeric
Processing ShT
Converted ShT to numeric
Processing Shot/90
Converted Shot/90 to numeric
Processing ShT/90
Converted ShT/90 to numeric
Processing Pens
Converted Pens to numeric
Processing Pens S
Converted Pens S to numeric
Processing Pen/R
Converted Pen/R to numeric
Processing Off
Converted Off to numeric
Processing xG
Converted xG to numeric
Processing CCC
Converted CCC to numeric
Processing Ch C/90
Converted Ch C/90 to numeric
Processing Ast
Converted Ast to numeric
Processing Asts/90
Converted Asts/90 t

In [231]:
# Check for NaN values in the 'UID' column
nan_rows = scouting_data[scouting_data['UID'].isna()]

# Display the rows with NaN values
print(nan_rows)


  Inf Name  UID Best Pos Best Role Best Duty Pros Cons Position Club  ... Cmd  \
0           NaN                                                       ... NaN   

  Bra Bal Ant Agi Agg Aer Acc decimal_height decimal_weight  
0 NaN NaN NaN NaN NaN NaN NaN            NaN            NaN  

[1 rows x 205 columns]


In [232]:
# Drop the NAN row totally
scouting_data = scouting_data.dropna(subset=['UID'])


In [233]:
# We want to make the UID numeric. We must clean the NaN above to resolve.
# Replace empty strings with NaN values
scouting_data['UID'] = scouting_data['UID'].replace('', np.nan)

# Set UID to large int
scouting_data['UID'] = scouting_data['UID'].astype('uint64') 

#scouting_data['UID'].max()

In [234]:
scouting_data['Expires']

1      7/31/2026
2      7/31/2027
3      7/31/2026
4      6/30/2026
5      6/30/2027
         ...    
252            -
253    7/31/2026
254    7/31/2026
255    7/31/2027
256            -
Name: Expires, Length: 256, dtype: object

In [235]:
#scouting_data['Best Duty'].unique()

In [236]:
# Convert Expires to datetime
# scouting_data['Expires']
# scouting_data['Salary']
# scouting_data['Dist/90']
# match_df['date'] = match_df['date'].apply(lambda x: datetime.strptime(x, "%a, %m/%d/%Y"))

# Replace "-" in 'Expires' column with NaN or something
# Replace empty strings with NaN values
# scouting_data['Expires'] = scouting_data['Expires'].replace(np.nan, '')
# scouting_data['Expires'] = scouting_data['Expires'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"))
# scouting_data['Expires']

scouting_data['Expires'] = scouting_data.apply(lambda row: determine_expires_date(row), axis=1)
scouting_data['Expires']


1     2026-07-31
2     2027-07-31
3     2026-07-31
4     2026-06-30
5     2027-06-30
         ...    
252          NaT
253   2026-07-31
254   2026-07-31
255   2027-07-31
256          NaT
Name: Expires, Length: 256, dtype: datetime64[ns]

In [237]:
scouting_data['Dist/90']

1          -
2      0.0mi
3          -
4      0.0mi
5      0.0mi
       ...  
252        -
253        -
254        -
255        -
256    0.0mi
Name: Dist/90, Length: 256, dtype: object

In [238]:
scouting_data['Dist/90'].unique()

array(['-', '0.0mi'], dtype=object)

In [171]:
# Remove any "mi" chars in all distance cols
# First convert to strings
scouting_data['Dist/90'] = scouting_data['Dist/90'].astype(str)
scouting_data['Distance'] = scouting_data['Distance'].astype(str)

# Then replace part of the string we wart removed "mi" --> ""
scouting_data['Dist/90'] = scouting_data['Dist/90'].str.replace('mi', '')
scouting_data['Distance'] = scouting_data['Distance'].str.replace('mi', '')

# Then coerce into numeric.
convert_to_numeric(scouting_data, scouting_data['Dist/90'])
convert_to_numeric(scouting_data, scouting_data['Distance'])


Converted Dist/90 to numeric
Converted Distance to numeric


In [175]:
scouting_data['Distance']

1       NaN
2      33.6
3       NaN
4      53.4
5      47.8
       ... 
252     NaN
253     NaN
254     NaN
255     NaN
256     6.0
Name: Distance, Length: 256, dtype: float64

In [184]:
scouting_data['Distance']

1       NaN
2      33.6
3       NaN
4      53.4
5      47.8
       ... 
252     NaN
253     NaN
254     NaN
255     NaN
256     6.0
Name: Distance, Length: 256, dtype: float64

In [182]:
scouting_data['Distance'].unique()

array([ nan, 33.6, 53.4, 47.8, 16.7,  0. , 61.1, 27.7, 32.7, 29.5, 14.4,
        5.8, 23.7, 14.8,  1.9,  6.6, 14.2,  2.4,  6.9,  9.9,  1.4, 24.4,
       59.8, 28.8, 13.1, 12.5, 17.8, 19.4, 13.4, 19. , 14. , 12.9, 65.2,
       19.8, 15.7,  2.3,  2.5,  9.5, 22.8,  5.4,  0.2, 24. , 29.8, 15.1,
        9.6, 15.5, 27.3, 17.1, 33.9,  6.8,  0.5, 13.7, 15.6,  7.6,  1.1,
        8.2,  6.3,  5.9,  3.5,  7.2,  7.3,  5.7, 40.8, 46.2,  5. ,  2.6,
       31.4,  4.7,  4.3, 38.1,  3.8,  8.1, 15.8, 33.8,  7. ,  7.9, 10.1,
       11.3, 16.5, 21.4, 39.1, 20.5, 10. , 24.1,  2.2,  9.7, 18.5, 14.9,
       15.2,  2. , 47. , 52.2,  0.4, 11.1, 16. ,  1.2,  1.3,  3.6, 12.1,
        0.7, 12.6, 11.8,  3.4, 28.1,  6. ])

In [204]:
# Convert all '%' cols to numeric
percent_cols = []
for col in scouting_data.columns:
    if re.search('%', col):
        print(col)
        percent_cols.append(col)
    # if re.match('%', col):
    #     print(f"Col {col}")
percent_cols
# foo = scouting_data.apply(lambda row: convert_percent_string_to_decimal(row, 'Conv %'), axis=1)

Shot %
Pas %
OP-Cr %
Conv %
Hdr %
xSv %
Sv %


['Shot %', 'Pas %', 'OP-Cr %', 'Conv %', 'Hdr %', 'xSv %', 'Sv %']

In [ ]:
# Iterate over all the percentage columns and change to a decimal value
for col in percent_cols:
    scouting_data[col] = scouting_data.apply(lambda row: convert_percent_string_to_decimal(row, col), axis=1)


In [206]:
scouting_data['Shot %']

1       NaN
2      0.38
3       NaN
4      0.29
5      0.00
       ... 
252     NaN
253     NaN
254     NaN
255     NaN
256     NaN
Name: Shot %, Length: 256, dtype: float64

In [208]:
scouting_data['Dist/90']

1      NaN
2      0.0
3      NaN
4      0.0
5      0.0
      ... 
252    NaN
253    NaN
254    NaN
255    NaN
256    0.0
Name: Dist/90, Length: 256, dtype: float64

In [207]:
# Needs openpyxl
# scouting_data.to_excel('data.xlsx', index=False)
scouting_data.to_csv('data.csv', index=False) 

In [61]:
# Convert the excluded columns to text
for col in EXCLUDE_COLS:
    

name_col = scouting_data['Name']
# Convert 'col1' to string
# df['col1'] = df['col1'].astype(str)
name_col.astype(str)

0                      
1           Martial Tia
2        Ibrahim Sulley
3         Mohamed Hamdy
4          George Ayine
             ...       
252     Sylvain Gbohouo
253     Moustapha Fanne
254        Mukhtar Moro
255    Derrick Fordjour
256     Haschou Kerrido
Name: Name, Length: 257, dtype: object